In [1]:
import pandas as pd
df = pd.read_parquet("./../../data/raw/cop26_tweets_en.parquet")

In [2]:
#!pip install bertopic==0.16.4
from bertopic import BERTopic
model  = BERTopic.load("./../../models/cop26_tweets_en.parquet.topic_model")

/home/arnaldo/Documents/prog-projects/complexity72_25/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-07-12 21:00:52,492 - BERTopic - WARNING: You are loading a BERTopic model without explicitly defining an embedding model. If you want to also load in an embedding model, make sure to use `BERTopic.load(my_model, embedding_model=my_embedding_model)`.


In [5]:
import pandas as pd
import numpy as np

ur_df = pd.read_parquet('./../../data/raw/cop26_tweets_en.parquet')
doc_info = pd.read_csv('./../../data/processed/document_info_cop26_tweets_en.csv')[['Document', 'Topic', 'Representative_document','Name']]
embeddings = np.load('./../../data/processed/cop26_tweets_en.parquet.npy')

In [6]:
print(doc_info.columns)

Index(['Document', 'Topic', 'Representative_document', 'Name'], dtype='object')


In [7]:
# merge ur_df with doc_info on 'text'
merged_df = ur_df.merge(doc_info, left_on='text', right_on='Document', how='left')
assert len(merged_df) == len(ur_df), 'differing length before and after merge, duplicates in right'

In [38]:
ur_df.columns

Index(['id', 'author_id', 'created_at', 'lang', 'in_reply_to_user_id',
       'conversation_id', 'text', 'reply_settings', 'possibly_sensitive',
       'retweeted_id', 'quoted_id', 'replied_id', 'url', 'expanded_url',
       'mention_name', 'hashtags', 'retweet_count', 'reply_count',
       'like_count', 'quote_count', 'username', 'individual_or_org',
       'category'],
      dtype='object')

In [12]:
# Function to select dominant topics
from collections import Counter

# Sample columns: 'author_id', 'i', 'Topic'
# df = pd.DataFrame(...)

def get_user_dominant_topic(topics, threshold=0.5, exclude=-1):
    """
    Args:
        topics (list): list of topic labels for a user.
        threshold (float): minimum fraction (e.g., 0.5 for 50%) for dominance.
        exclude (int or list): topic(s) to exclude, e.g. -1.
    
    Returns:
        int or None: dominant topic if above threshold, else None
    """
    if isinstance(exclude, int):
        topics = [t for t in topics if t != exclude]
    elif isinstance(exclude, list):
        topics = [t for t in topics if t not in exclude]

    if not topics:
        return None

    total = len(topics)
    topic_counts = Counter(topics)
    top_topic, top_count = topic_counts.most_common(1)[0]
    
    if top_count / total >= threshold:
        return top_topic
    return None


In [14]:
user_topics = merged_df.groupby('author_id')['Topic'].apply(list)

# Try with 30% threshold
user_dominant_topic_30 = user_topics.apply(lambda topics: get_user_dominant_topic(topics, threshold=0.3))

# Try with 50% threshold
user_dominant_topic_50 = user_topics.apply(lambda topics: get_user_dominant_topic(topics, threshold=0.5))

# Try with 60% threshold
user_dominant_topic_60 = user_topics.apply(lambda topics: get_user_dominant_topic(topics, threshold=0.6))

# Combine into a DataFrame for comparison
result = pd.DataFrame({
    'dominant_topic_30': user_dominant_topic_30,
    'dominant_topic_50': user_dominant_topic_50,
    'dominant_topic_60': user_dominant_topic_60
})